In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkifyETL_Test") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2,com.amazonaws:aws-java-sdk-bundle:1.11.563") \
    .getOrCreate()

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.aws.credentials.provider", "com.amazonaws.auth.profile.ProfileCredentialsProvider")
hadoop_conf.set("aws.profile", "default")

spark.sparkContext.setLogLevel("ERROR")


Define Table Paths

In [8]:
output_data = "s3a://sparkify-datalake-aws/curated/"

songs_path = output_data + "songs/"
artists_path = output_data + "artists/"
users_path = output_data + "users/"
time_path = output_data + "time/"
songplays_path = output_data + "songplays/"


Load & Preview Each Table

In [9]:
songs_df = spark.read.parquet(songs_path)
artists_df = spark.read.parquet(artists_path)
users_df = spark.read.parquet(users_path)
time_df = spark.read.parquet(time_path)
songplays_df = spark.read.parquet(songplays_path)


In [10]:
# Show schemas
songs_df.printSchema()
artists_df.printSchema()
users_df.printSchema()
time_df.printSchema()
songplays_df.printSchema()


root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

root
 |-- user_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- level: string (nullable = true)

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

root
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable

In [ ]:
# Preview rows
songs_df.show(5)
artists_df.show(5)
users_df.show(5)
time_df.show(5)
songplays_df.show(5)


+------------------+--------------------+---------+----+------------------+
|           song_id|               title| duration|year|         artist_id|
+------------------+--------------------+---------+----+------------------+
|SOAOIBZ12AB01815BE|I Hold Your Hand ...| 43.36281|2000|ARPBNLO1187FB3D52F|
|SONYPOM12A8C13B2D7|I Think My Wife I...|186.48771|2005|ARDNS031187B9924F0|
|SODREIN12A58A7F2E5|A Whiter Shade Of...|326.00771|   0|ARLTWXK1187FB5A3F8|
|SOYMRWW12A6D4FAB14|The Moon And I (O...| 267.7024|   0|ARKFYS91187B98E58F|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|279.97995|   0|ARPFHN61187FB575F6|
+------------------+--------------------+---------+----+------------------+
only showing top 5 rows

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     97|      Kate|  Harrell|     F| paid|
|     26|      Ryan|    Smith|     M| free|
|     43|    Jahiem|    Miles|     M| free|
|     63|      Ayla|  Johns

Basic Validations

In [14]:
# Check row counts
print("🎵 Songs:", songs_df.count())
print("🎨 Artists:", artists_df.count())
print("👥 Users:", users_df.count())
print("⏰ Time:", time_df.count())
print("▶️ Songplays:", songplays_df.count())


🎵 Songs: 71
🎨 Artists: 69
👥 Users: 104
⏰ Time: 6813
▶️ Songplays: 6820


Quick Analytics


In [15]:
# Most played songs
songplays_df.groupBy("song_id").count().orderBy("count", ascending=False).show(10)

# Top users
songplays_df.groupBy("user_id").count().orderBy("count", ascending=False).show(10)

# Plays by time of day
time_df.groupBy("hour").count().orderBy("hour").show()


+------------------+-----+
|           song_id|count|
+------------------+-----+
|              NULL| 6816|
|SOGDBUF12A8C140FAA|    3|
|SOZCTXZ12AB0182364|    1|
+------------------+-----+

+-------+-----+
|user_id|count|
+-------+-----+
|     49|  689|
|     80|  665|
|     97|  557|
|     15|  463|
|     44|  397|
|     29|  346|
|     24|  321|
|     73|  289|
|     88|  270|
|     36|  248|
+-------+-----+
only showing top 10 rows

+----+-----+
|hour|count|
+----+-----+
|   0|  157|
|   1|  189|
|   2|  179|
|   3|  207|
|   4|  253|
|   5|  314|
|   6|  322|
|   7|  314|
|   8|  330|
|   9|  442|
|  10|  470|
|  11|  517|
|  12|  490|
|  13|  506|
|  14|  385|
|  15|  367|
|  16|  271|
|  17|  229|
|  18|  200|
|  19|  156|
+----+-----+
only showing top 20 rows

